# Media Info

> Functions for getting media information and managing metadata

In [ ]:
#| default_exp media_info

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from typing import Dict, Optional
from datetime import datetime
import json
import os
import subprocess
import tempfile

## Media Duration

In [ ]:
#| export
def get_media_duration(file_path: Path # Path to the media file
                      ) -> Optional[float]: # Duration in seconds, or None if duration cannot be determined
    """
    Get the duration of a media file using ffprobe.
    """
    duration_cmd = [
        'ffprobe',
        '-v', 'quiet',
        '-show_entries', 'format=duration',
        '-of', 'csv=p=0',
        str(file_path)
    ]
    
    try:
        duration_result = subprocess.run(
            duration_cmd,
            check=True,
            capture_output=True,
            text=True
        )
        return float(duration_result.stdout.strip())
    except (subprocess.CalledProcessError, ValueError, FileNotFoundError):
        return None

## Audio Information

In [ ]:
#| export
def get_audio_info_ffmpeg(
    file_path: Path  # Path to audio file
) -> Optional[Dict[str, any]]: # Dictionary containing audio info, or None if extraction fails
    """Get basic audio file information including title if available."""
    
    cmd = [
        'ffprobe',
        '-v', 'quiet',
        '-show_entries', 
        'stream=codec_type,codec_name,duration,sample_rate,channels,tags:format=duration,tags:format_tags',
        '-show_format',
        '-show_streams',
        '-of', 'json',
        str(file_path)
    ]
        
    try:
        result_subprocess = subprocess.run(
            cmd,
            check=True,
            capture_output=True,
            text=True
        )
        
        probe_data = json.loads(result_subprocess.stdout)
        
        # Find the audio stream
        audio_stream = next((stream for stream in probe_data.get('streams', []) 
                            if stream.get('codec_type') == 'audio'), None)
        
        processed_result = {}
        
        if audio_stream:
            # Get duration from stream, fallback to format duration
            duration = float(audio_stream.get('duration', 
                            probe_data.get('format', {}).get('duration', 0)))
            sample_rate = int(audio_stream.get('sample_rate', 0))
            channels = int(audio_stream.get('channels', 0))
            
            processed_result.update({
                'duration_seconds': duration,
                'sample_rate': sample_rate,
                'channels': channels,
                'format': file_path.suffix.lower() if hasattr(file_path, 'suffix') else '',
                'codec': audio_stream.get('codec_name'),
                'method': 'subprocess'
            })
            
            # Extract title from ffmpeg metadata
            # Check both format tags and stream tags
            all_tags = {}
            
            # Get format tags
            format_info = probe_data.get('format', {})
            if 'tags' in format_info:
                all_tags.update(format_info['tags'])
            
            # Get stream tags (some metadata might be stored here)
            if audio_stream and 'tags' in audio_stream:
                all_tags.update(audio_stream['tags'])

            # Debug: print available tags
            # print(f"Available tags: {list(all_tags.keys())}")
            
            if all_tags:
                # Try different common title tag names (case-insensitive)
                title_keys = ['title', 'Title', 'TITLE', 'TIT2']
                for title_key in title_keys:
                    # Check for exact match first
                    if title_key in all_tags:
                        processed_result['title'] = all_tags[title_key]
                        break
                else:
                    # If no exact match, do case-insensitive search
                    tags_lower = {k.lower(): v for k, v in all_tags.items()}
                    for title_key in title_keys:
                        if title_key.lower() in tags_lower:
                            processed_result['title'] = tags_lower[title_key.lower()]
                            break
                    # If no title found in metadata, use filename without extension as fallback
                    if 'title' not in processed_result:
                        processed_result['title'] = file_path.stem

                
        else:
            processed_result = None
            
    except (subprocess.CalledProcessError, ValueError, FileNotFoundError, json.JSONDecodeError):
        processed_result = None

    return processed_result

## Metadata Management

In [ ]:
#| export
def add_metadata_with_ffmpeg(
    input_file: Path, # Path to input audio file
    metadata: Dict[str, str] # Dictionary of metadata key-value pairs
) -> bool: # True if metadata was added successfully, False otherwise
    """Add metadata to audio file using FFmpeg."""
    # Create temporary file
    temp_fd, temp_path = tempfile.mkstemp(suffix='.wav', dir=input_file.parent)
    os.close(temp_fd)
    
    cmd = ['ffmpeg', '-i', input_file]
    
    # Add metadata parameters
    for key, value in metadata.items():
        cmd.extend(['-metadata', f'{key}={value}'])
    
    # Output to temporary file
    cmd.extend(['-c', 'copy', '-y', temp_path])
    
    try:
        subprocess.run(cmd, check=True, capture_output=True)
        
        # Replace original file with temporary file
        os.replace(temp_path, input_file)
        print(f"Metadata added to {input_file}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error adding metadata: {e}")
        # Clean up temp file if it exists
        if os.path.exists(temp_path):
            os.remove(temp_path)
        return False

## Testing

In [ ]:
#|eval: false
# Example usage of metadata functions
from datetime import datetime

test_audio_path = Path("test_audio.wav")
if test_audio_path.exists():
    metadata = {
        'title': 'Test Audio',
        'artist': 'Python Audio Generator',
        'album': 'Test Album',
        'date': f'{datetime.today().year}',
        'genre': 'Test',
        'comment': 'Test audio file with metadata'
    }
    
    add_metadata_with_ffmpeg(test_audio_path, metadata)
    info = get_audio_info_ffmpeg(test_audio_path)
    if info:
        print(f"Audio info: {info}")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()